# VGG16 for Age Estimation

## Import Data

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
#import images from google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import glob
bucket0 = glob.glob('/content/drive/MyDrive/StocktonImagesBucketed/0/*.*')
bucket1 = glob.glob('/content/drive/MyDrive/StocktonImagesBucketed/1/*.*')
bucket2 = glob.glob('/content/drive/MyDrive/StocktonImagesBucketed/2/*.*')
bucket3 = glob.glob('/content/drive/MyDrive/StocktonImagesBucketed/3/*.*')
bucket4 = glob.glob('/content/drive/MyDrive/StocktonImagesBucketed/4/*.*')
bucket5 = glob.glob('/content/drive/MyDrive/StocktonImagesBucketed/5/*.*')
bucket6 = glob.glob('/content/drive/MyDrive/StocktonImagesBucketed/6/*.*')
bucket7 = glob.glob('/content/drive/MyDrive/StocktonImagesBucketed/7/*.*')
bucket8 = glob.glob('/content/drive/MyDrive/StocktonImagesBucketed/8/*.*')

data = []
labels = []
for i in bucket0:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('0')

for i in bucket1:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('1')

for i in bucket2:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('2')      

for i in bucket3:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('3')

for i in bucket4:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('4')

for i in bucket5:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('5')   

for i in bucket6:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('6')

for i in bucket7:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('7')

for i in bucket8:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append('8')        

data = np.array(data)
labels = np.array(labels)

In [ ]:
#Split data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels,       
test_size=0.1)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
#Normalize data
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

#Convert labels to categorical
from keras.utils import to_categorical
y_train = to_categorical(y_train, 9)
y_test = to_categorical(y_test, 9)

## Model

In [3]:
from keras.applications import VGG16
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [4]:
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [5]:
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model

x = vgg_model.output
x = Flatten()(x) # Flatten dimensions to for use in FC layers
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
x = Dense(256, activation='relu')(x)
x = Dense(9, activation='softmax')(x) # Softmax for multiclass
transfer_model = Model(inputs=vgg_model.input, outputs=x)

In [11]:
# Freeze four convolution blocks
for layer in transfer_model.layers[:15]:
    layer.trainable = False

# Make sure you have frozen the correct layers
for i, layer in enumerate(transfer_model.layers):
    print(i, layer.name, layer.trainable)

0 input_2 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False
19 flatten True
20 dense True
21 dropout True
22 dense_1 True
23 dense_2 True


In [8]:
from keras import optimizers

learning_rate= 0.001
transfer_model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=learning_rate), metrics=["accuracy"])
history = transfer_model.fit(X_train, y_train, batch_size = 16, epochs=50, validation_data=(X_test,y_test))

In [ ]:
#Plot accuracy and loss
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Transfer Learning Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()